# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol
import psycopg2

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [4]:
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')

# Load
- 등록&제원 -> result 4등급(연료 : 경유, 휘발유, LPG)

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [5]:
# 8.6s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")

In [6]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898814 entries, 0 to 1898813
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   VIN             object 
 1   BSPL_STDG_CD    int64  
 2   VHCL_ERSR_YN    object 
 3   MANP_MNG_NO     object 
 4   YRIDNW          int64  
 5   VHCTY_CD        object 
 6   PURPS_CD2       object 
 7   FRST_REG_YMD    int64  
 8   VHCL_FBCTN_YMD  float64
 9   VHCL_MNG_NO     object 
dtypes: float64(1), int64(3), object(6)
memory usage: 144.9+ MB


In [7]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호', 
}

In [8]:
carr = car.rename(columns=car_ch_col)

In [9]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [10]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")

In [11]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
}

In [12]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [13]:
# 3.0s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010946 entries, 0 to 1010945
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   VIN           1010946 non-null  object
 1   RDCDVC_SE_CD  1010946 non-null  object
dtypes: object(2)
memory usage: 15.4+ MB


In [14]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}

In [15]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [16]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       47916 non-null  int64 
 1   STDG_CTPV_NM  47916 non-null  object
 2   STDG_SGG_NM   47892 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [17]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [18]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [19]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## 등록이력(CEG_CAR_HISTORY_MIG)

In [20]:
# 1m 13.2s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

In [21]:
# # 1.8s
# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()
# sql = 'select VHCL_ERSR_YN, CHNG_DE, VHMNO from vsysd.ceg_car_history_mig'
# cur.execute(sql)
# his = pd.DataFrame(cur.fetchall())
# his.columns = [desc[0].upper() for desc in cur.description]
# cur.close()
# conn.close()

## RH에서 제공한 법정동코드

In [22]:
rh = we.export_to_pandas("SELECT DONG_CODE, CTPRVN_NM, SIGNGU_NM FROM TB_MAPDATA;")
rh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DONG_CODE  278 non-null    int64 
 1   CTPRVN_NM  278 non-null    object
 2   SIGNGU_NM  260 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


In [23]:
rh = rh.rename(columns={'DONG_CODE':'법정동코드_rh', 'CTPRVN_NM':'시도', 'SIGNGU_NM':'시군구'})
rh.head()

,법정동코드_rh,시도,시군구
0,0,전국,NaN
1,1100000000,서울특별시,NaN
2,1111000000,서울특별시,종로구
3,1114000000,서울특별시,중구
4,1117000000,서울특별시,용산구


In [24]:
rh.shape

(278, 3)

## 4등급 result(for DPF유무)

In [25]:
rs = we.export_to_pandas("SELECT 차대번호, DPF유무_수정 FROM GRD4_RESULT;")
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


# 전처리

## 중복 차대번호 제거

In [26]:
carr.shape, len(carr['차대번호'].unique())

((1898814, 10), 1898763)

In [27]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1898763, 10)

## 차종 코드 변환

In [28]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승합', '승용', '특수', nan], dtype=object)

## 용도 코드 변환

In [29]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [30]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1515737, 10)

## 연료 코드 변환

In [31]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', 'CNG 하이브리드', '등유', '경유 하이브리드', '태양열',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [32]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [33]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1515737, 12)

## 저감장치구분 코드 변환

In [34]:
# 4m 14.3s
attr['저감장치구분'].unique()

array(['A1001', 'A1003', 'A1005', 'A1006', 'A1002', 'A1004', 'A1007'],
      dtype=object)

In [35]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '엔진교체', '2종', '1종+SCR', '삼원촉매'], dtype=object)

## 저감장치 부착 유무

In [36]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [37]:
attr['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    531833
유      479113
Name: count, dtype: int64

## 등록&제원&저감이력 병합

In [38]:
# 1.7s
csa = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
csa.shape

(1515737, 13)

## 지역정보 병합

In [39]:
df = csa.merge(coder, on='법정동코드', how='left')

### 매칭 안되는 지역 처리

In [40]:
# 경기도 양주시
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

### rh 법정동코드 참고하여 법정동코드 수정

In [41]:
df.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구'],
      dtype='object')

In [42]:
rdf = df.copy()

In [43]:
rdf['법정동코드'] = rdf['법정동코드'].astype('str')
rdf['법정동코드_mod'] = rdf['법정동코드'].str[:5] + '00000'
rdf['법정동코드_mod'] = pd.to_numeric(rdf['법정동코드_mod'])
rdf['법정동코드_mod'].head()

0    4824000000
1    4482500000
2    4518000000
3    2826000000
4    2771000000
Name: 법정동코드_mod, dtype: int64

In [44]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

{2817000000, 4119500000, 4119700000, 4119900000, 4173000000, 4371000000}

In [45]:
# 2817000000 인천광역시 남구 -> 인천광역시 미추홀구 2817700000
# 4119500000 경기도 부천시 원미구 -> 경기도 부천시 4119000000
# 4119700000 경기도 부천시 소사구 -> 경기도 부천시 4119000000
# 4119900000 경기도 부천시 오정구 -> 경기도 부천시 4119000000
# 4173000000 경기도 여주군 -> 경기도 여주시 4167000000
# 4371000000 충청북도 청원군 -> 충청북도 충주시 4311000000

In [46]:
rdf.loc[rdf['법정동코드_mod'] == 2817000000, '법정동코드_mod'] = 2817700000
rdf.loc[rdf['법정동코드_mod'] == 4119500000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119700000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119900000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4173000000, '법정동코드_mod'] = 4167000000
rdf.loc[rdf['법정동코드_mod'] == 4371000000, '법정동코드_mod'] = 4311000000

In [47]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

set()

In [48]:
rdf.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod'],
      dtype='object')

In [49]:
rdf['법정동코드_mod'].isnull().sum()

0

In [50]:
df = rdf.copy()
df.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod'],
      dtype='object')

## 4등급 result 파일 참고하여 DPF유무 수정

In [51]:
rdf = df.copy()

In [52]:
len(set(rdf['차대번호'].unique()) - set(rs['차대번호'].unique()))

53185

In [53]:
rdf['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    1515676
유           61
Name: count, dtype: int64

In [54]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879481
NaN     348422
유       261897
확인불가     12435
Name: count, dtype: int64

In [55]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

(1502213, 2)

In [56]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879481
NaN     348400
유       261897
확인불가     12435
Name: count, dtype: int64

In [57]:
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [58]:
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'

In [59]:
rdf1['DPF_YN'].value_counts(dropna=False)

DPF_YN
무       839836
NaN     401576
유       261892
확인불가     12433
Name: count, dtype: int64

In [60]:
rdf1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod',
       'DPF유무_수정'],
      dtype='object')

In [61]:
df = rdf1.drop('DPF유무_수정', axis=1)

In [62]:
df.shape

(1515737, 16)

## 연료 확인

In [63]:
df['연료'].value_counts(dropna=False)

연료
경유             1156240
휘발유             333206
LPG(액화석유가스)      26179
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: count, dtype: int64

## 연료 컬럼 추가

In [64]:
df1 = df.copy()

In [65]:
df1.loc[df1['연료'] == '경유', 'fuel'] = '경유'
df1.loc[(df1['연료'] == '휘발유') | (df1['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'

# 분석

## EG 분류

In [66]:
df1['fuel'].value_counts(dropna=False)

fuel
경유        1156240
휘발유_가스     359385
NaN           112
Name: count, dtype: int64

In [67]:
grade_list = []
for f, y, cy, e in tqdm(df1[['fuel', '제작일자', '차량연식', 'DPF_YN']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 1515737/1515737 [00:02<00:00, 507557.87it/s]


1515737

In [68]:
df1['Grade'] = grade_list

In [69]:
df1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod',
       'fuel', 'Grade'],
      dtype='object')

In [70]:
df2 = df1[[
    '차대번호', 
    '제원관리번호',
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도', 
    '시군구', 
    'DPF_YN',
    'Grade',
    '법정동코드_mod',
    ]]
df2.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드', '시도',
       '시군구', 'DPF_YN', 'Grade', '법정동코드_mod'],
      dtype='object')

In [71]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230714'

In [72]:
df2['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_16988\2409603865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['테이블생성일자'] = today_date


In [73]:
df2.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드', '시도',
       '시군구', 'DPF_YN', 'Grade', '법정동코드_mod', '테이블생성일자'],
      dtype='object')

In [74]:
df3 = df2[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구', 
    'DPF_YN', 
    'Grade', 
    '법정동코드_mod',
    ]]
df3.columns

Index(['테이블생성일자', '차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드',
       '시도', '시군구', 'DPF_YN', 'Grade', '법정동코드_mod'],
      dtype='object')

In [75]:
df3['Grade'].value_counts(dropna=False)

Grade
C    800869
B    387925
A    215684
X     72292
D     38967
Name: count, dtype: int64

In [76]:
df3['법정동코드_mod'] = df3['법정동코드_mod'].astype('str')

In [77]:
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO',
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2',
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM',
                '시군구':'SGG_NM',
                'DPF_YN':'DPF_MNTNG_YN',
                'Grade':'GRD4_MLSFC', 
                '법정동코드_mod':'STDG_CD_MOD'
                }

In [78]:
STD_BD_GRD4_MLSFC_RSLT = df3.rename(columns=ch_col_dict)
STD_BD_GRD4_MLSFC_RSLT.columns

Index(['LOAD_DT', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2', 'YRIDNW',
       'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'DPF_MNTNG_YN',
       'GRD4_MLSFC', 'STDG_CD_MOD'],
      dtype='object')

In [79]:
STD_BD_GRD4_MLSFC_RSLT.tail()

,LOAD_DT,VIN,MANG_MNG_NO,VHCTY_CD,PURPS_CD2,YRIDNW,VHCTY_TY,FUEL_CD,STDG_CD,CTPV_NM,SGG_NM,DPF_MNTNG_YN,GRD4_MLSFC,STDG_CD_MOD
1515732,20230714,KNAD2322JS055503,10015800000000,승용,개인용,1995,소형,휘발유,4155025029,경기도,안성시,NaN,B,4155000000
1515733,20230714,KNADA2322HS006936,10015800000000,승용,개인용,1987,소형,휘발유,5011013700,제주특별자치도,제주시,NaN,B,5011000000
1515734,20230714,2T1CF2897YC359923,99931107024351303,승용,개인용,2000,대형,휘발유,1117012400,서울특별시,용산구,NaN,A,1117000000
1515735,20230714,SL68T032460,10017500140000,승용,개인용,1968,대형,휘발유,1165010100,서울특별시,서초구,NaN,B,1165000000
1515736,20230714,CN-62-T-2213,30110100002636,승용,개인용,2002,소형,휘발유,1159010200,서울특별시,동작구,NaN,X,1159000000


## [출력] STD_BD_GRD4_MLSFC_RSLT

In [80]:
expdf = STD_BD_GRD4_MLSFC_RSLT
table_nm = 'STD_BD_GRD4_MLSFC_RSLT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

# 분석2

In [81]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

In [82]:
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

## 등록 & 제원 정보 병합(말소 유지)

In [83]:
carr.shape

(1898763, 10)

In [84]:
csersr = carr.merge(srcr[['제원관리번호', '연료']], on='제원관리번호', how='left')
csersr.shape

(1898763, 11)

## 1\. 차량관리번호 기준 병합

In [85]:
csersr.shape

(1898763, 11)

In [86]:
# 58.3s
ersr = csersr.merge(hisr, on='차량관리번호', how='left')
ersr.shape

(4063732, 13)

## 2\. 차량말소YN 만 추출

In [87]:
errm = ersr[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)
errm.shape

(369841, 13)

## 3\. 변경일자 최신으로 차대번호 중복 제거

In [88]:
errm = errm.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

## 4\. 변경일자 2019.01.01 이상만 추출

In [89]:
errm = errm[errm['변경일자'] >= 20190101].reset_index(drop=True)

In [90]:
errm.shape

(367089, 13)

In [91]:
errm.columns

Index(['차대번호', '법정동코드', '차량말소YN_x', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차량말소YN_y', '변경일자'],
      dtype='object')

## 5\. 법정동 코드 정보 병합

In [92]:
errc = errm.merge(coder, on='법정동코드', how='left')

## 시도, 연도별 차량 대수

In [93]:
df['최초등록일자'].describe()

count    1515737.00
mean    20056364.03
std        66403.08
min     19650209.00
25%     20060622.00
50%     20081111.00
75%     20100510.00
max     20220614.00
Name: 최초등록일자, dtype: float64

In [94]:
dfm = df.copy()

In [95]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')

In [96]:
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

### 시군구명 앞쪽 지역명만 남기기(dfm)

In [97]:
# 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

### 연료 지역별 등록차량대수

In [121]:
num_car_by_local = dfm.groupby(['연료', '시도', '시군구_수정', '최초등록일자_년', '최초등록일자_월'], as_index=False)['차대번호'].count()
num_car_by_local = num_car_by_local.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도', '최초등록일자_월':'월'})
num_car_by_local

,연료,시도,시군구_수정,연도,월,등록차량대수
0,CNG(압축천연가스),전라남도,신안군,2005,04,1
1,LPG(액화석유가스),강원도,강릉시,1990,03,1
2,LPG(액화석유가스),강원도,강릉시,1992,04,1
3,LPG(액화석유가스),강원도,강릉시,1993,05,1
4,LPG(액화석유가스),강원도,강릉시,1993,07,1
...,...,...,...,...,...,...
75670,휘발유,충청북도,충주시,2009,02,1
75671,휘발유,충청북도,충주시,2009,03,1
75672,휘발유,충청북도,충주시,2010,03,1
75673,휘발유,충청북도,충주시,2013,05,1


### 연료 지역별 차량대수

In [105]:
dfm['최초등록일자'].max()

'20220614'

In [106]:
num_car_by_local2 = dfm.loc[dfm['최초등록일자'] < dfm['최초등록일자'].max()].groupby(['연료', '시도', '시군구_수정'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'차량대수'})
num_car_by_local2

,연료,시도,시군구_수정,차량대수
0,CNG(압축천연가스),전라남도,신안군,1
1,LPG(액화석유가스),강원도,강릉시,67
2,LPG(액화석유가스),강원도,고성군,15
3,LPG(액화석유가스),강원도,동해시,184
4,LPG(액화석유가스),강원도,삼척시,21
...,...,...,...,...
814,NaN,전라북도,군산시,3
815,NaN,전라북도,익산시,1
816,NaN,충청북도,옥천군,1
817,NaN,충청북도,진천군,1


In [107]:
dfm['최초등록일자_년'].max(), dfm['최초등록일자_월'].max()

('2022', '12')

In [108]:
num_car_by_local2[['연도', '월']] = [dfm['최초등록일자_년'].max(), dfm['최초등록일자_월'].max()]
num_car_by_local2

,연료,시도,시군구_수정,차량대수,연도,월
0,CNG(압축천연가스),전라남도,신안군,1,2022,12
1,LPG(액화석유가스),강원도,강릉시,67,2022,12
2,LPG(액화석유가스),강원도,고성군,15,2022,12
3,LPG(액화석유가스),강원도,동해시,184,2022,12
4,LPG(액화석유가스),강원도,삼척시,21,2022,12
...,...,...,...,...,...,...
814,NaN,전라북도,군산시,3,2022,12
815,NaN,전라북도,익산시,1,2022,12
816,NaN,충청북도,옥천군,1,2022,12
817,NaN,충청북도,진천군,1,2022,12


### 연료 지역별 말소 대수

In [109]:
errc['변경일자'].head()

0   20220628.00
1   20220628.00
2   20220628.00
3   20220628.00
4   20220628.00
Name: 변경일자, dtype: float64

In [110]:
errc['변경일자'] = errc['변경일자'].astype('str')

In [111]:
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

### 시군구명 앞쪽 지역명만 남기기(errc)

In [112]:
# 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [113]:
grp_erase = errc.loc[errc['변경일자_년'] == dfm['최초등록일자_년'].max()].groupby(['변경일자_년', '변경일자_월', '연료', '시도', '시군구_수정'], as_index=False)['차대번호'].count()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도', '변경일자_월':'월'})
grp_erase = grp_erase.sort_values(['시도', '시군구_수정'])
grp_erase

,연도,월,연료,시도,시군구_수정,말소차량대수
0,2022,01,LPG(액화석유가스),강원도,강릉시,1
106,2022,01,경유,강원도,강릉시,26
353,2022,01,휘발유,강원도,강릉시,11
671,2022,02,경유,강원도,강릉시,38
917,2022,02,휘발유,강원도,강릉시,6
...,...,...,...,...,...,...
2256,2022,04,휘발유,충청북도,충주시,6
2609,2022,05,경유,충청북도,충주시,47
2829,2022,05,휘발유,충청북도,충주시,7
3168,2022,06,경유,충청북도,충주시,44


In [114]:
num_car_by_local2['연도'].max()

'2022'

In [115]:
yr_list = []
mth_list = []
fuel_list = []
ctpv_list = []
sgg_list = []
yr = int(num_car_by_local2['연도'].max())
mth = int(num_car_by_local2['월'].max())
sl = num_car_by_local2.drop_duplicates(['시도', '시군구_수정']).reset_index(drop=True)
for ctpv, sgg in sl[['시도', '시군구_수정']].values:
    for fuel in sl['연료'].unique():
        for mth in range(mth - 4, mth + 1):
            mthm = f'{mth:0>2}'
            yr_list.append(str(yr))
            mth_list.append(mthm)
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
            sgg_list.append(sgg)
base = pd.DataFrame({'연도':yr_list, '월':mth_list, '연료':fuel_list, '시도':ctpv_list, '시군구_수정':sgg_list})

In [116]:
base

,연도,월,연료,시도,시군구_수정
0,2022,08,CNG(압축천연가스),전라남도,신안군
1,2022,09,CNG(압축천연가스),전라남도,신안군
2,2022,10,CNG(압축천연가스),전라남도,신안군
3,2022,11,CNG(압축천연가스),전라남도,신안군
4,2022,12,CNG(압축천연가스),전라남도,신안군
...,...,...,...,...,...
5175,2022,08,휘발유,인천광역시,남구
5176,2022,09,휘발유,인천광역시,남구
5177,2022,10,휘발유,인천광역시,남구
5178,2022,11,휘발유,인천광역시,남구


In [117]:
num_car_by_local2.head()

,연료,시도,시군구_수정,차량대수,연도,월
0,CNG(압축천연가스),전라남도,신안군,1,2022,12
1,LPG(액화석유가스),강원도,강릉시,67,2022,12
2,LPG(액화석유가스),강원도,고성군,15,2022,12
3,LPG(액화석유가스),강원도,동해시,184,2022,12
4,LPG(액화석유가스),강원도,삼척시,21,2022,12


In [122]:
base1 = base.merge(num_car_by_local2, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [123]:
base2 = base1.merge(num_car_by_local, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [124]:
base3 = base2.merge(grp_erase, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [125]:
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3[['차량대수', '등록차량대수', '말소차량대수']].isnull().sum()

차량대수      0
등록차량대수    0
말소차량대수    0
dtype: int64

In [126]:
n = len(base3['월'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']
    # base3.loc[(i+1)*n - 3, '차량대수'] = base3.loc[(i+1)*n - 2, '차량대수'] + base3.loc[(i+1)*n - 2, '말소차량대수'] - base3.loc[(i+1)*n - 2, '등록차량대수']
    # base3.loc[(i+1)*n - 4, '차량대수'] = base3.loc[(i+1)*n - 3, '차량대수'] + base3.loc[(i+1)*n - 3, '말소차량대수'] - base3.loc[(i+1)*n - 3, '등록차량대수']
    # base3.loc[(i+1)*n - 5, '차량대수'] = base3.loc[(i+1)*n - 4, '차량대수'] + base3.loc[(i+1)*n - 4, '말소차량대수'] - base3.loc[(i+1)*n - 4, '등록차량대수']

In [128]:
base3

,연도,월,연료,시도,시군구_수정,차량대수,등록차량대수,말소차량대수
0,2022,08,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
1,2022,09,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
2,2022,10,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
3,2022,11,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
4,2022,12,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...
5175,2022,08,휘발유,인천광역시,남구,10.00,0.00,0.00
5176,2022,09,휘발유,인천광역시,남구,10.00,0.00,0.00
5177,2022,10,휘발유,인천광역시,남구,10.00,0.00,0.00
5178,2022,11,휘발유,인천광역시,남구,10.00,0.00,0.00


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

In [ ]:
base3['테이블생성일자'] = today_date

In [ ]:
base4 = base3[[
    '테이블생성일자', 
    '연도',
    '월', 
    '연료', 
    '시도', 
    '시군구_수정', 
    '차량대수',
]]

In [ ]:
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM',
    '연도':'YR', 
    '월':'MM', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    '차량대수':'VHCL_MKCNT', 
}

In [ ]:
STD_BD_GRD4_RGN_CURSTT = base4.rename(columns=chc_col)
STD_BD_GRD4_RGN_CURSTT.columns

### [출력] STD_BD_GRD4_RGN_CURSTT

In [ ]:
# expdf = STD_BD_GRD4_RGN_CURSTT
# table_nm = 'STD_BD_GRD4_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# 분석3

In [129]:
dfm.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod',
       '최초등록일자_년', '최초등록일자_월', '최초등록일자_일', '시군구_수정'],
      dtype='object')

In [130]:
errc.columns

Index(['차대번호', '법정동코드', '차량말소YN_x', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차량말소YN_y', '변경일자', '시도', '시군구', '변경일자_년',
       '변경일자_월', '변경일자_일', '시군구_수정'],
      dtype='object')

## 연도 시도 차종별 차량 대수

### 현재 차량 대수

In [ ]:
num_car_by_local1 = dfm.groupby(['연료', '시도', '차종'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})
num_car_by_local1

In [132]:
num_car_by_local1['연도'] = dfm['최초등록일자_년'].max()
num_car_by_local1

,연료,시도,차종,차량대수,연도
0,CNG(압축천연가스),전라남도,화물,1,2022
1,LPG(액화석유가스),강원도,승용,458,2022
2,LPG(액화석유가스),강원도,승합,153,2022
3,LPG(액화석유가스),강원도,화물,188,2022
4,LPG(액화석유가스),경기도,승용,3342,2022
...,...,...,...,...,...
194,NaN,서울특별시,승합,2,2022
195,NaN,인천광역시,승용,3,2022
196,NaN,전라남도,승용,1,2022
197,NaN,전라북도,승용,4,2022


### 등록 차량 대수

In [133]:
num_car_by_local2 = dfm.groupby(['연료', '시도', '차종', '최초등록일자_년'], as_index=False)['차대번호'].count()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도'})
num_car_by_local2

,연료,시도,차종,연도,등록차량대수
0,CNG(압축천연가스),전라남도,화물,2005,1
1,LPG(액화석유가스),강원도,승용,1989,3
2,LPG(액화석유가스),강원도,승용,1990,4
3,LPG(액화석유가스),강원도,승용,1991,13
4,LPG(액화석유가스),강원도,승용,1992,5
...,...,...,...,...,...
2353,휘발유,충청북도,화물,1995,8
2354,휘발유,충청북도,화물,1996,5
2355,휘발유,충청북도,화물,1997,2
2356,휘발유,충청북도,화물,1998,2


### 말소 차량 대수

In [134]:
grp_erase = errc.groupby(['변경일자_년', '연료', '시도', '차종'], as_index=False)['차대번호'].count()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도'})
grp_erase = grp_erase.sort_values(['시도'])
grp_erase

,연도,연료,시도,차종,말소차량대수
0,2019,CNG(압축천연가스),강원도,화물,1
461,2021,휘발유,강원도,승용,856
396,2021,경유,강원도,화물,1160
395,2021,경유,강원도,특수,16
394,2021,경유,강원도,승합,411
...,...,...,...,...,...
288,2020,경유,충청북도,승합,343
287,2020,경유,충청북도,승용,1689
222,2020,LPG(액화석유가스),충청북도,화물,28
626,2022,경유,충청북도,특수,6


In [135]:
yr_list = []
fuel_list = []
ctpv_list = []
cd_list = []
yr = int(num_car_by_local2['연도'].max())
for ctpv in num_car_by_local1['시도'].unique():
    for fuel in sl['연료'].unique():
        for cd in ['승용', '승합', '화물', '특수']:
            for yrm in range(yr - 3, yr + 1):
                yr_list.append(str(yrm))
                fuel_list.append(fuel)
                ctpv_list.append(ctpv)
                cd_list.append(cd)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '시도':ctpv_list, '차종':cd_list})

In [136]:
base

,연도,연료,시도,차종
0,2019,CNG(압축천연가스),전라남도,승용
1,2020,CNG(압축천연가스),전라남도,승용
2,2021,CNG(압축천연가스),전라남도,승용
3,2022,CNG(압축천연가스),전라남도,승용
4,2019,CNG(압축천연가스),전라남도,승합
...,...,...,...,...
1083,2022,휘발유,충청북도,화물
1084,2019,휘발유,충청북도,특수
1085,2020,휘발유,충청북도,특수
1086,2021,휘발유,충청북도,특수


In [137]:
base1 = base.merge(num_car_by_local1, on=['연도', '연료', '시도', '차종'], how='left')
base1

,연도,연료,시도,차종,차량대수
0,2019,CNG(압축천연가스),전라남도,승용,NaN
1,2020,CNG(압축천연가스),전라남도,승용,NaN
2,2021,CNG(압축천연가스),전라남도,승용,NaN
3,2022,CNG(압축천연가스),전라남도,승용,NaN
4,2019,CNG(압축천연가스),전라남도,승합,NaN
...,...,...,...,...,...
1083,2022,휘발유,충청북도,화물,153.00
1084,2019,휘발유,충청북도,특수,NaN
1085,2020,휘발유,충청북도,특수,NaN
1086,2021,휘발유,충청북도,특수,NaN


In [138]:
base2 = base1.merge(num_car_by_local2, on=['연도', '연료', '시도', '차종'], how='left')
base2

,연도,연료,시도,차종,차량대수,등록차량대수
0,2019,CNG(압축천연가스),전라남도,승용,NaN,NaN
1,2020,CNG(압축천연가스),전라남도,승용,NaN,NaN
2,2021,CNG(압축천연가스),전라남도,승용,NaN,NaN
3,2022,CNG(압축천연가스),전라남도,승용,NaN,NaN
4,2019,CNG(압축천연가스),전라남도,승합,NaN,NaN
...,...,...,...,...,...,...
1083,2022,휘발유,충청북도,화물,153.00,NaN
1084,2019,휘발유,충청북도,특수,NaN,NaN
1085,2020,휘발유,충청북도,특수,NaN,NaN
1086,2021,휘발유,충청북도,특수,NaN,NaN


In [139]:
base3 = base2.merge(grp_erase, on=['연도', '연료', '시도', '차종'], how='left')
base3

,연도,연료,시도,차종,차량대수,등록차량대수,말소차량대수
0,2019,CNG(압축천연가스),전라남도,승용,NaN,NaN,NaN
1,2020,CNG(압축천연가스),전라남도,승용,NaN,NaN,NaN
2,2021,CNG(압축천연가스),전라남도,승용,NaN,NaN,NaN
3,2022,CNG(압축천연가스),전라남도,승용,NaN,NaN,NaN
4,2019,CNG(압축천연가스),전라남도,승합,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1083,2022,휘발유,충청북도,화물,153.00,NaN,3.00
1084,2019,휘발유,충청북도,특수,NaN,NaN,NaN
1085,2020,휘발유,충청북도,특수,NaN,NaN,NaN
1086,2021,휘발유,충청북도,특수,NaN,NaN,NaN


In [140]:
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3[['차량대수', '등록차량대수', '말소차량대수']].isnull().sum()

차량대수      0
등록차량대수    0
말소차량대수    0
dtype: int64

In [141]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']
    # base3.loc[(i+1)*n - 3, '차량대수'] = base3.loc[(i+1)*n - 2, '차량대수'] + base3.loc[(i+1)*n - 2, '말소차량대수'] - base3.loc[(i+1)*n - 2, '등록차량대수']
    # base3.loc[(i+1)*n - 4, '차량대수'] = base3.loc[(i+1)*n - 3, '차량대수'] + base3.loc[(i+1)*n - 3, '말소차량대수'] - base3.loc[(i+1)*n - 3, '등록차량대수']
    # base3.loc[(i+1)*n - 5, '차량대수'] = base3.loc[(i+1)*n - 4, '차량대수'] + base3.loc[(i+1)*n - 4, '말소차량대수'] - base3.loc[(i+1)*n - 4, '등록차량대수']

In [145]:
base3.tail(20)

,연도,연료,시도,차종,차량대수,등록차량대수,말소차량대수
1068,2019,경유,충청북도,특수,333.00,1.00,13.00
1069,2020,경유,충청북도,특수,323.00,0.00,10.00
1070,2021,경유,충청북도,특수,306.00,0.00,17.00
1071,2022,경유,충청북도,특수,300.00,0.00,6.00
1072,2019,휘발유,충청북도,승용,13604.00,0.00,1003.00
1073,2020,휘발유,충청북도,승용,12181.00,0.00,1423.00
1074,2021,휘발유,충청북도,승용,11237.00,0.00,944.00
1075,2022,휘발유,충청북도,승용,10907.00,0.00,330.00
1076,2019,휘발유,충청북도,승합,116.00,0.00,6.00
1077,2020,휘발유,충청북도,승합,112.00,0.00,4.00


In [ ]:
today_date = datetime.today().strftime("%Y%m")
today_date

In [ ]:
base3['테이블생성일자'] = today_date

In [ ]:
base4 = base3[[
    '테이블생성일자', 
    '연도', 
    '연료', 
    '시도', 
    '차종', 
    '차량대수', 
]]

In [ ]:
chc_col = {
    '테이블생성일자':'LOAD_DT',
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '차량대수':'VHCL_MKCNT', 
}

In [ ]:
STD_BD_GRD4_RGN_CURSTT_MOD = base4.rename(columns=chc_col)
STD_BD_GRD4_RGN_CURSTT_MOD.columns

In [ ]:
STD_BD_GRD4_RGN_CURSTT_MOD.head()

### [출력] STD_BD_GRD4_RGN_CURSTT_MOD

In [ ]:
# expdf = STD_BD_GRD4_RGN_CURSTT_MOD
# table_nm = 'STD_BD_GRD4_RGN_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# 코드 마지막